In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from shapely import wkt

import geopandas as gpd
import pandas as pd
from shapely import wkt
import json
import pandas as pd
import osmnx as ox
import matplotlib.pyplot as plt
import osmnx as ox
import networkx as nx
import GOSTnets as gn
import logging
import datetime

import itertools
from multiprocessing import Pool, cpu_count
import multiprocessing

/home/leibnitz.rojas/.local/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
##Logger
logging.root.handlers.clear()
logging.root.handlers

# timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# log_filename = f"logs/CuscoEdgesWaze1400_{timestamp}.log"

# logging.basicConfig(
#     level=logging.INFO,
#     datefmt='%Y-%m-%d %H:%M:%S',
# )

logger = logging.getLogger()
logger.setLevel(logging.INFO)

logFormatter = logging.Formatter('%(asctime)s :: %(levelname)s :: %(name)s :: %(process)d :: %(message)s')

# fileHandler = logging.FileHandler(log_filename)
# fileHandler.setFormatter(logFormatter)
# logger.addHandler(fileHandler)

streamHandler = logging.StreamHandler()
streamHandler.setFormatter(logFormatter)
logger.addHandler(streamHandler)
##Logger

In [3]:
mapaLimaCallao = pd.read_csv('../SimBig22/rutas/MapaLimaCallao.csv')
mapaLimaCallao['geometry'] = mapaLimaCallao['geometry'].apply(wkt.loads)
mapaLimaCallao = gpd.GeoDataFrame(mapaLimaCallao, crs='epsg:4326')
mapaLimaCallao['distrito2'] = mapaLimaCallao.apply(lambda x: x['distrito'].replace(' ','_'), axis=1)
mapaLimaCallao.head()

,iddpto,departamen,idprov,provincia,iddist,distrito,capital,codccpp,area,fuente,geometry,LimaCallao,distrito2
0,7,CALLAO,701,CALLAO,70105,LA PUNTA,LA PUNTA,1,1,INEI,"POLYGON ((-77.15884 -12.06645, -77.15874 -12.0...",True,LA_PUNTA
1,7,CALLAO,701,CALLAO,70104,LA PERLA,LA PERLA,1,1,INEI,"POLYGON ((-77.10444 -12.07020, -77.10486 -12.0...",True,LA_PERLA
2,7,CALLAO,701,CALLAO,70102,BELLAVISTA,BELLAVISTA,1,1,INEI,"POLYGON ((-77.10086 -12.06307, -77.10317 -12.0...",True,BELLAVISTA
3,7,CALLAO,701,CALLAO,70103,CARMEN DE LA LEGUA REYNOSO,CARMEN DE LA LEGUA REYNOSO,1,1,INEI,"POLYGON ((-77.08317 -12.03925, -77.08263 -12.0...",True,CARMEN_DE_LA_LEGUA_REYNOSO
4,7,CALLAO,701,CALLAO,70101,CALLAO,CALLAO,1,1,INEI,"POLYGON ((-77.11219 -11.97352, -77.11203 -11.9...",True,CALLAO


In [4]:
lima_nodes = pd.read_csv("data/GrafoLima_nodes.csv", index_col=[0])
lima_nodes['geometry'] = lima_nodes['geometry'].apply(wkt.loads)
lima_nodes = gpd.GeoDataFrame(lima_nodes, crs='epsg:4326')

lima_edges = pd.read_csv("data/GrafoLima_edgeswaze.csv", index_col=[0,1,2])
lima_edges['geometry'] = lima_edges['geometry'].apply(wkt.loads)
lima_edges = gpd.GeoDataFrame(lima_edges, crs='epsg:4326')

/home/leibnitz.rojas/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (5,7,13,14,15,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
lima_edges.head()

osmid  oneway lanes  \
u         v           key                                                     
258066779 10985400003 0                              39457621    True     1   
          258066780   0    [1046533254, 338193454, 143630423]    True     2   
258066780 258066781   0                             143630430    True     2   
258066781 258066783   0                             143630430    True     2   
          495302399   0                              40752940    True     1   

                                 highway maxspeed    length  \
u         v           key                                     
258066779 10985400003 0    motorway_link       80   109.492   
          258066780   0         motorway      100   782.922   
258066780 258066781   0            trunk      100  1301.315   
258066781 258066783   0            trunk      100   194.871   
          495302399   0       trunk_link      NaN    85.154   

                                                                    geometry  \
u         v           key                                                      
258066779 10985400003 0    LINESTRING (-76.78468 -12.37136, -76.78466 -12...   
          258066780   0    LINESTRING (-76.78468 -12.37136, -76.78433 -12...   
258066780 258066781   0    LINESTRING (-76.77956 -12.37632, -76.77670 -12...   
258066781 258066783   0    LINESTRING (-76.77105 -12.38455, -76.76977 -12...   
          495302399   0    LINESTRING (-76.77105 -12.38455, -76.77099 -12...   

                             ref                        name bridge  ...  \
u         v           key                                            ...   
258066779 10985400003 0      NaN                         NaN    NaN  ...   
          258066780   0    PE-1S  Carretera Panamericana Sur    yes  ...   
258066780 258066781   0    PE-1S  Carretera Panamericana Sur    NaN  ...   
258066781 258066783   0    PE-1S  Carretera Panamericana Sur    NaN  ...   
          495302399   0      NaN                         NaN    NaN  ...   

                          access tunnel width  service  \
u         v           key                                
258066779 10985400003 0      NaN    NaN   NaN      NaN   
          258066780   0      NaN    NaN   NaN      NaN   
258066780 258066781   0      NaN    NaN   NaN      NaN   
258066781 258066783   0      NaN    NaN   NaN      NaN   
          495302399   0      NaN    NaN   NaN      NaN   

                                           wazeinfo0800  \
u         v           key                                 
258066779 10985400003 0                   (0.05, 0.107)   
          258066780   0                   (0.45, 0.781)   
258066780 258066781   0    (0.03333333333333333, 0.063)   
258066781 258066783   0     (1.6666666666666667, 0.612)   
          495302399   0                    (0.1, 0.091)   

                                           wazeinfo1400 wazeinfo0800_minutes  \
u         v           key                                                      
258066779 10985400003 0                   (0.05, 0.107)             0.050000   
          258066780   0                   (0.45, 0.781)             0.450000   
258066780 258066781   0    (0.03333333333333333, 0.063)             0.033333   
258066781 258066783   0     (1.7166666666666666, 0.612)             1.666667   
          495302399   0                    (0.1, 0.091)             0.100000   

                           wazeinfo0800_km  wazeinfo1400_minutes  \
u         v           key                                          
258066779 10985400003 0              0.107              0.050000   
          258066780   0              0.781              0.450000   
258066780 258066781   0              0.063              0.033333   
258066781 258066783   0              0.612              1.716667   
          495302399   0              0.091              0.100000   

                           wazeinfo1400_km  
u         v           key                   
258066779 1098540

# Grafo Lima

In [34]:
G = ox.graph_from_gdfs(lima_nodes, lima_edges)
print(nx.info(G))

Name: 
Type: MultiDiGraph
Number of nodes: 148111
Number of edges: 395620
Average in degree:   2.6711
Average out degree:   2.6711


In [35]:
H = G.to_undirected()

In [36]:
print(nx.info(H))

Name: 
Type: MultiGraph
Number of nodes: 148111
Number of edges: 222783
Average degree:   3.0083


In [ ]:
# H_isolates = list(nx.isolates(H))
# H_isolates

In [ ]:
# print("Number of nodes: %s" % H.number_of_nodes())
# print("Number of isolates %s" % nx.number_of_isolates(H))

# print("Ratio of isolates %.3f" % (nx.number_of_isolates(H) / H.number_of_nodes()) )

In [ ]:
# H.remove_nodes_from(H_isolates)

In [ ]:
# print(nx.info(H))
# nx.number_of_isolates(H)

# Lima Amenities

In [6]:
%%time
place_name = "Lima Metropolitana"
# graph = ox.graph_from_place(place_name, which_result=3, network_type='drive')
amenitiesLima = ox.geometries_from_place(place_name,tags={'amenity':True} )

CPU times: user 34.7 s, sys: 1.15 s, total: 35.8 s
Wall time: 36.1 s


In [7]:
from shapely.geometry import Point
amenitiesLima['isPoint'] = amenitiesLima.apply(lambda x: type(x.geometry) is Point , axis=1)
amenitiesLima.rename(columns={'geometry':'geometry_polygon'}, inplace=True)
amenitiesLima['geometry'] = amenitiesLima.apply(lambda x: x['geometry_polygon'].centroid, axis=1)
amenitiesLima.set_geometry('geometry')
amenitiesLima.set_index('osmid', inplace=True)
amenitiesLima.head(2)

,unique_id,element_type,amenity,geometry_polygon,fee,wheelchair,wheelchair:description,addr:city,addr:housenumber,addr:street,...,bicycle_rental,trolleybus,fuel:octane_87,internet,wikipedia:en,wikipedia:pl,wikipedia:sv,shortest_name,isPoint,geometry
osmid,,,,,,,,,,,,,,,,,,,,,
1706421372,node/1706421372,node,bus_station,POINT (-76.90809 -12.25501),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,POINT (-76.90809 -12.25501)
1909452321,node/1909452321,node,toilets,POINT (-76.90717 -12.25586),no,no,No hay baño para silla de ruedas y encima tien...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,POINT (-76.90717 -12.25586)


In [8]:
cols = ['bank', 'school', 'police', 'hospital', 'university', 'courthouse']
amenities = amenitiesLima.query("amenity in @cols")

In [9]:
amenities['amenity'].unique()

array(['police', 'bank', 'school', 'courthouse', 'university', 'hospital'],
      dtype=object)

In [12]:
amenities

,unique_id,element_type,amenity,geometry_polygon,fee,wheelchair,wheelchair:description,addr:city,addr:housenumber,addr:street,...,bicycle_rental,trolleybus,fuel:octane_87,internet,wikipedia:en,wikipedia:pl,wikipedia:sv,shortest_name,isPoint,geometry
osmid,,,,,,,,,,,,,,,,,,,,,
3259019072,node/3259019072,node,police,POINT (-76.82449 -12.33707),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,POINT (-76.82449 -12.33707)
3759247695,node/3759247695,node,bank,POINT (-76.90608 -12.21089),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,POINT (-76.90608 -12.21089)
3759586688,node/3759586688,node,bank,POINT (-76.91030 -12.23102),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,POINT (-76.91030 -12.23102)
4235492500,node/4235492500,node,school,POINT (-76.78649 -12.49113),NaN,NaN,NaN,NaN,NaN,Entrada A Pucusana,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,POINT (-76.78649 -12.49113)
4235507693,node/4235507693,node,school,POINT (-76.79644 -12.48408),NaN,NaN,NaN,NaN,NaN,Callao,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,POINT (-76.79644 -12.48408)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117746162,way/1117746162,way,school,"POLYGON ((-77.04981 -11.86239, -77.05026 -11.8...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,POINT (-77.04983 -11.86293)
1117749137,way/1117749137,way,school,"POLYGON ((-77.07248 -11.86274, -77.07202 -11.8...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,POINT (-77.07181 -11.86272)
1184113689,way/1184113689,way,police,"POLYGON ((-76.96830 -11.72886, -76.96832 -11.7...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,POINT (-76.96827 -11.72893)


In [15]:
%%time
logger.info("Proceso SNAP Inicio")
amenities_snap_limanodes = gn.pandana_snap_points(amenities, lima_nodes)
logger.info("Proceso SNAP Fin")

2024-02-13 21:53:30,938 :: INFO :: root :: 11378 :: Proceso SNAP Inicio
/home/leibnitz.rojas/.local/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/leibnitz.rojas/.local/lib/python3.6/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/home/leibnitz.rojas/.local/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' s

CPU times: user 6h 48min 45s, sys: 4min 10s, total: 6h 52min 55s
Wall time: 6h 52min 50s


In [17]:
amenities_snap_limanodes.rename(columns={'NN':'glima_osmid', 'NN_dist':'glima_dist' }, inplace=True)

In [18]:
amenities_snap_limanodes.head()

,unique_id,element_type,amenity,geometry_polygon,fee,wheelchair,wheelchair:description,addr:city,addr:housenumber,addr:street,...,fuel:octane_87,internet,wikipedia:en,wikipedia:pl,wikipedia:sv,shortest_name,isPoint,geometry,glima_osmid,glima_dist
osmid,,,,,,,,,,,,,,,,,,,,,
3259019072,node/3259019072,node,police,POINT (-76.82449 -12.33707),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,POINT (-76.82449 -12.33707),495244831,0.000330
3759247695,node/3759247695,node,bank,POINT (-76.90608 -12.21089),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,POINT (-76.90608 -12.21089),3759247684,0.000133
3759586688,node/3759586688,node,bank,POINT (-76.91030 -12.23102),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,True,POINT (-76.91030 -12.23102),3756733899,0.000346
4235492500,node/4235492500,node,school,POINT (-76.78649 -12.49113),NaN,NaN,NaN,NaN,NaN,Entrada A Pucusana,...,NaN,NaN,NaN,NaN,NaN,NaN,True,POINT (-76.78649 -12.49113),1831352656,0.078740
4235507693,node/4235507693,node,school,POINT (-76.79644 -12.48408),NaN,NaN,NaN,NaN,NaN,Callao,...,NaN,NaN,NaN,NaN,NaN,NaN,True,POINT (-76.79644 -12.48408),1932467999,0.073871


# Calculando Ruta más cercana a un amenity

In [46]:
amenities_snap_limanodes['amenity'].unique()

array(['police', 'bank', 'school', 'courthouse', 'university', 'hospital'],
      dtype=object)

In [63]:
var_amenity = 'bank'

logger.info(f"Starting process for amenity {var_amenity} ")
cond = amenities_snap_limanodes['amenity'] == var_amenity

nearestnodes2amenities = list(amenities_snap_limanodes.loc[cond,:]['glima_osmid'].values)
nearestnodes2amenities = list(set(nearestnodes2amenities))
len(nearestnodes2amenities)

2024-02-15 21:32:12,710 :: INFO :: root :: 11378 :: Starting process for amenity bank 


839

In [64]:
# fig, ax = plt.subplots(figsize=(15,15))

# # nodes.plot(ax=ax, color='g')
# # edges.plot(ax=ax, color='yellow', alpha=0.5)
# lima_edges.plot(ax=ax)
# # cuscoshape.plot(ax=ax, color='gray', alpha=0.2)
# # rutascuscounique_gnodes.plot(ax=ax) 
# # inputdf.plot(ax=ax, color='red')

# # amenities_cols = ['unique_id', 'osmid', 'element_type', 'amenity', 'name',  'geometry',]
# # amenities.loc[cond, amenities_cols ].head(2)
# amenities_snap_limanodes.loc[cond, :].plot(ax=ax, color='green', marker='x' , )
# # amenities_gtransporte.loc[[874430122], :].plot(ax=ax, color='black', )

# lima_nodes.loc[nearestnodes2amenities,:].plot(ax=ax, color='red')

# # nodes.loc[nodes2plot,:].plot(ax=ax, color='red')
# # H_nodes.loc[[30965748],:].plot(ax=ax, color='black')
# # H_nodes.loc[[995446222],:].plot(ax=ax, color='cyan')
# # 30965748	995446222

In [65]:
nodes_source = list(set(lima_nodes.index))
print(len(nodes_source))
nodes_target = nearestnodes2amenities
print(len(nodes_target))

148111
839


In [66]:
nodes_sourcecrosstarget = list(itertools.product(nodes_source, nodes_target))
len(nodes_sourcecrosstarget)

124265129

In [70]:
def shortest_path_transporte(G, node_source, node_target):
    try:
        resdict = {}
        for w in ['wazeinfo0800_minutes', 'wazeinfo0800_km', 'wazeinfo1400_minutes', 'wazeinfo1400_km']:
#         for w in ['tiempoponderado30kmh']:
            res = nx.shortest_path_length(G, 
                                      source=node_source, 
                                      target=node_target, 
                                      weight=w)
            resdict[w] = res
    except Exception as e:
        resdict = {}
    return resdict  

In [68]:
nodes_sourcecrosstarget_df = pd.DataFrame(nodes_sourcecrosstarget, columns=['source', 'target'])
nodes_sourcecrosstarget_df.head(3)

,source,target
0,1738539013,108191745
1,1738539013,4410796033
2,1738539013,108191747


In [69]:
nodes_sourcecrosstarget_df.shape[0] , nodes_sourcecrosstarget_df.shape[0]/40

(124265129, 3106628.225)

In [ ]:
%%time
def process_row(row):
    result = shortest_path_transporte(H, row.source, row.target)
    return result

def split_dataframe(df, chunk_size):
    chunks = [df[i:i + chunk_size] for i in range(0, df.shape[0], chunk_size)]
    return chunks

def apply_chunk(chunk):
    return chunk.apply(process_row, axis=1)

# Define the chunk size based on your dataset size and available resources
chunk_size = 500000

inputdf = nodes_sourcecrosstarget_df

data_chunks = split_dataframe(inputdf, chunk_size)

# Get the number of available CPU cores
# num_cores = cpu_count()
num_cores = int(inputdf.shape[0]/chunk_size)+1
if num_cores > cpu_count():
    num_cores = cpu_count()
# num_cores = cpu_count()
print(f"Num Cores --> {num_cores}")

# # Create a multiprocessing pool with the number of cores
with Pool(num_cores) as pool:
    logger.info("Multiprocessing started.")
    results = pool.map(apply_chunk, data_chunks)
    logger.info("Multiprocessing completed.")

Num Cores --> 40


2024-02-15 21:36:35,272 :: INFO :: root :: 11378 :: Multiprocessing started.


In [ ]:
results_df = pd.DataFrame(pd.concat(results), columns=['dict'])
results_df.head()

In [ ]:
cond = results_df['dict'].map(len)>0
results_df[cond]

In [ ]:
df = nodes_sourcecrosstarget_df.merge(results_df[cond], how='inner', left_index=True, right_index=True)
df.head()

In [ ]:
%%time
# Create an empty DataFrame with columns as unique keys from all dictionaries
columns = set().union(*(d.keys() for d in df['dict'] if d))
df2 = pd.DataFrame(columns=columns)
df2.head()

In [ ]:
%%time
# Iterate through each row's dictionary and populate the result DataFrame
for idx, row in df.iterrows():
    df2.loc[idx] = row['dict']

In [ ]:
# df.merge(df2, )
df3 = df[['source', 'target']].merge(df2, how='left', left_index=True, right_index=True)
df3

In [ ]:
df3.to_csv(f"data/limaprivado_wazeresults_{var_amenity}.csv", sep=';', index=False)

logger.info(f"Finishing process for amenity {var_amenity} ")

In [ ]:
!ls data/